# 영산강 광산보 주별 데이터, RNN 적용

**필수 라이브러리**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

**matplotlib 한글 설정**

- 운영체제에 따른 한글 지원 설정. 윈도우, 우분투, 구글 코랩 지원.
- 참고: [matplotlib에서 한글 지원하기](https://github.com/codingalzi/datapy/blob/master/matplotlib-korean.md)

In [ ]:
import platform

if platform.system() == 'Windows': # 윈도우
    from matplotlib import font_manager, rc
    font_path = "C:/Windows/Fonts/NGULIM.TTF"
    font = font_manager.FontProperties(fname=font_path).get_name()
    rc('font', family=font)
elif platform.system() == 'Linux': # 우분투 또는 구글 코랩
    # please run the following commented out codes just once
#     if 'google.colab' in str(get_ipython()):
#         !apt-get install -y fonts-nanum*
#     else:
#         !sudo apt-get install -y fonts-nanum*
#     !fc-cache -fv
    
    applyfont = "NanumBarunGothic"
    import matplotlib.font_manager as fm
    if not any(map(lambda ft: ft.name == applyfont, fm.fontManager.ttflist)):
        fm.fontManager.addfont("/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf")
    plt.rc("font", family=applyfont)
    plt.rc("axes", unicode_minus=False)
    

## 데이터 준비

**데이터 저장소**

데이터 원본 파일 저장소는 다음과 같다.

In [ ]:
base_url = "https://github.com/codingalzi/water-data/raw/master/reservoirs/"

**영산강(엑셀) 자료를 데이터프레임으로 불러오기**

모든 지역의 데이터 불러오기: 평동천, 광산, 장성천2, 문평천, 영산포2, 함평, 무안2

- `header=0`: 0번 행을 header로 지정, 즉 열 인덱스로 사용.
- `sheet_name=None`: 모든 워크시트 가져오기. 워크시트별로 하나의 df 생성. 반환값은 사전.
- `na_values=0`: 0으로 입력된 값도 결측치로 처리
- `index_col=1`: 측정일을 행 인덱스로 사용
- `parse_dates=True`: 행 인덱스로 사용되는 날짜 대상 파싱 실행

In [ ]:
# 주의: 엑셀파일을 불러오기 위해 아래 모듈이 필요하다.

# !pip install openpyxl

In [ ]:
yeongsan = pd.read_excel(base_url+"Yeongsan_weekly_total.xlsx",
                            header=0, 
                            sheet_name=None,
                            na_values=0,
                            index_col=1, 
                            parse_dates=True)

포함된 보(reservoir)의 지역명은 다음과 같다.

In [ ]:
locations = yeongsan.keys()
locations

dict_keys(['1_평동천', '2_광산', '3_장성천2', '4_문평천', '5_영산포2', '6_함평', '7_무안2'])

**주요 특성**

수온, BOD, COD, TN, TP, 유량 등 6개의 주요 특성만을 이용하여 클로로필-A 예측하려 한다.
원 데이터셋에 포함된 19개의 특성은 다음과 같다.

In [ ]:
list(yeongsan['1_평동천'].columns)

['측정소명',
 '회차',
 '수온(℃)',
 'DO(㎎/L)',
 'BOD(㎎/L)',
 'COD(㎎/L)',
 '클로로필 a(㎎/㎥)',
 'TN(㎎/L)',
 'TP(㎎/L)',
 'TOC(㎎/L)',
 '수소이온농도',
 '전기전도도(μS/㎝)',
 '용존총질소(㎎/L)',
 '암모니아성 질소(㎎/L)',
 '질산성 질소(㎎/L)',
 '용존총인(㎎/L)',
 '인산염인(㎎/L)',
 'SS(㎎/L)',
 '유량(㎥/s)']

주요 특성 6개와 타깃으로 사용될 특성인 클로로필-A를 별도로 지정한다.

In [ ]:
features_important = ['수온(℃)', 'BOD(㎎/L)', 'COD(㎎/L)', 'TN(㎎/L)', 'TP(㎎/L)', '유량(㎥/s)', '클로로필 a(㎎/㎥)']

- 입력데이터셋 특성 6개

In [ ]:
six_features = features_important[:6]

* 타깃 특성: 클로로필-A

In [ ]:
target_feature = features_important[-1]

**지역별 데이터**

지역별 데이터셋 크기는 다음과 같다.

In [ ]:
total_data = 0

for loc in locations:
    ys_loc = yeongsan[loc]
    total_data += ys_loc.shape[0]
    print(f"{loc}: \t{ys_loc.shape}")
    
print("총 데이터수:", total_data)

1_평동천: 	(440, 19)
2_광산: 	(510, 19)
3_장성천2: 	(435, 19)
4_문평천: 	(435, 19)
5_영산포2: 	(456, 19)
6_함평: 	(456, 19)
7_무안2: 	(534, 19)
총 데이터수: 3266


**지역별 데이터 분포**

모든 지역에서 유량과 클로로필-A 특성에서 일부 결측지가 존재한다.

- 광산 데이터셋이 가장 크면서 가장 적은 유량 결측치를 가진다.

In [ ]:
for loc in locations:
    print(f"\n=== {loc} ===\n")
    ys_loc = yeongsan[loc][features_important]
    ys_loc.info()



=== 1_평동천 ===

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 440 entries, 2012-06-05 to 2022-06-27
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   수온(℃)        440 non-null    float64
 1   BOD(㎎/L)     440 non-null    float64
 2   COD(㎎/L)     440 non-null    float64
 3   TN(㎎/L)      440 non-null    float64
 4   TP(㎎/L)      440 non-null    float64
 5   유량(㎥/s)      437 non-null    float64
 6   클로로필 a(㎎/㎥)  424 non-null    float64
dtypes: float64(7)
memory usage: 27.5 KB

=== 2_광산 ===

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 510 entries, 2012-01-04 to 2021-12-20
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   수온(℃)        510 non-null    float64
 1   BOD(㎎/L)     510 non-null    float64
 2   COD(㎎/L)     510 non-null    float64
 3   TN(㎎/L)      510 non-null    float64
 4   TP(㎎/L)      510 non-null    float64
 5   유량(㎥/

**광산 지역 데이터**

In [ ]:
kwangsan = yeongsan['2_광산'][features_important]

In [ ]:
kwangsan.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 510 entries, 2012-01-04 to 2021-12-20
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   수온(℃)        510 non-null    float64
 1   BOD(㎎/L)     510 non-null    float64
 2   COD(㎎/L)     510 non-null    float64
 3   TN(㎎/L)      510 non-null    float64
 4   TP(㎎/L)      510 non-null    float64
 5   유량(㎥/s)      508 non-null    float64
 6   클로로필 a(㎎/㎥)  510 non-null    float64
dtypes: float64(7)
memory usage: 31.9 KB


결측치를 포함한 데이터 2개는 제거한다.

In [ ]:
kwangsan = kwangsan.dropna()

**특성 정규화**

모든 특성을 정규화한다. 

먼저 특성별 평균값과 표준편차를 계산한다.

In [ ]:
kwangsan_mean = kwangsan.mean(axis=0)
kwangsan_std = kwangsan.std(axis=0)

평균은 0, 표준편차는 1로 변환한다.

In [ ]:
kwangsan = (kwangsan - kwangsan_mean)/kwangsan_std
kwangsan

,수온(℃),BOD(㎎/L),COD(㎎/L),TN(㎎/L),TP(㎎/L),유량(㎥/s),클로로필 a(㎎/㎥)
년/월/일,,,,,,,
2012-01-04,-1.669834,-0.126796,-0.403615,1.064816,4.079187,-0.477342,-0.669352
2012-01-09,-1.441486,-0.190132,-0.246860,2.530872,6.795254,-0.426468,-0.669352
2012-01-18,-1.416114,1.076584,0.497731,3.238200,6.380299,-0.478248,0.421764
2012-01-27,-1.644462,0.379890,0.889620,2.218704,0.646380,-0.397462,0.657466
2012-01-30,-1.441486,0.886577,0.497731,1.588710,4.182925,-0.450120,0.098666
...,...,...,...,...,...,...,...
2021-11-22,-0.502721,-0.570147,-0.286049,-0.375304,-0.664499,-0.529255,0.040403
2021-11-29,-0.718383,-0.633483,-0.403615,0.371634,0.363457,-0.545875,-0.581957
2021-12-06,-1.010161,-0.063460,-0.756316,-0.173480,0.221995,-0.396050,-0.216486


## 시계열 데이터 분석

시간의 흐름을 고려해서 시계열(timeseries) 데이터로 처리한다.

**날짜별로 정렬**

먼저 날짜별로 정렬한다.

In [ ]:
kwangsan = kwangsan.sort_index()

**훈련셋과 테스트셋 지정**

훈련셋, 검증셋, 테스트셋을 7:2:1의 비율로 나눈다.
단, 테스트셋은 날짜를 기준으로 나중에 측정된 데이터를 이용한다.

In [ ]:
train_size = int(kwangsan.shape[0] * 0.7)
val_size = int(kwangsan.shape[0] * 0.2)

- 훈련셋

In [ ]:
train_set = kwangsan[six_features][:train_size]
train_targets = kwangsan[target_feature][:train_size]

- 검증셋

In [ ]:
val_set = kwangsan[six_features][train_size : train_size + val_size]
val_targets = kwangsan[target_feature][train_size : train_size + val_size]

- 테스트셋

In [ ]:
test_set = kwangsan[six_features][train_size + val_size:]
test_targets = kwangsan[target_feature][train_size + val_size:]

**시계열 데이터로 변환**

In [ ]:
import tensorflow as tf

시계열 데이터 샘플을 `sequence_length` 만큼의 타임 스텝(time step)로
구성한다.
예측값은 미래가 아닌 현재의 클로로필-A 수치로 지정한다.

- 타임 스텝 크기: 8(두 달)
- 배치 크기: 32
- `shuffle=True`: 구성된 시계열 샘플 뒤섞기

In [ ]:
sequence_length=8  # 타임 스텝 크기

train_dataset = tf.keras.utils.timeseries_dataset_from_array(
    train_set,
    targets=train_targets[sequence_length-1:],
    sequence_length=sequence_length,
    shuffle=True,
    batch_size=32)

val_dataset = tf.keras.utils.timeseries_dataset_from_array(
    val_set,
    targets=val_targets[sequence_length-1:],
    sequence_length=sequence_length,
    shuffle=True,
    batch_size=32)

test_dataset = tf.keras.utils.timeseries_dataset_from_array(
    test_set,
    targets=test_targets[sequence_length-1:],
    sequence_length=sequence_length,
    shuffle=True,
    batch_size=32)

입력 데이터셋의 배치의 모양은 `(32, sequence_length, 6)`이다.

In [ ]:
for samples, targets in train_dataset:
    print("samples shape:", samples.shape)
    print("targets shape:", targets.shape)
    break

samples shape: (32, 8, 6)
targets shape: (32,)


**GRU 모델 사용**

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

- 훈련 중 `dropout_rate` 만큼의 드랍아웃 적용
- 조기종료 실행. (검증셋에 대한 성능이 더 이상 좋아지지 않는 경우)
- 성능이 별로 좋지 않음.
- 보다 많은 특성을 사용하는 것이 나을 수도 있음

In [ ]:
dropout_rate = 0.5

inputs = keras.Input(shape=(sequence_length, len(features_important)-1))
x = layers.GRU(48, recurrent_dropout=dropout_rate, return_sequences=True)(inputs)
x = layers.GRU(48, recurrent_dropout=dropout_rate)(x)
x = layers.Dropout(dropout_rate)(x)
outputs = layers.Dense(1)(x)
model = keras.Model(inputs, outputs)
  
early_stopping_cb = keras.callbacks.EarlyStopping(
        monitor="val_mae", patience=20, restore_best_weights=True)

callbacks = [
    keras.callbacks.ModelCheckpoint("kwangsan_gru.keras",
                                    save_best_only=True),
    early_stopping_cb
]

model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])

history = model.fit(train_dataset,
                    epochs=100,
                    validation_data=val_dataset,
                    callbacks=callbacks)

model = keras.models.load_model("kwangsan_gru.keras") 

print(f"Test MAE: {model.evaluate(test_dataset)[1]:.2f}")

Epoch 1/100
11/11 [==============================] - 4s 88ms/step - loss: 0.6709 - mae: 0.6440 - val_loss: 1.3146 - val_mae: 0.8216
Epoch 2/100
11/11 [==============================] - 1s 55ms/step - loss: 0.5898 - mae: 0.6059 - val_loss: 1.2644 - val_mae: 0.8072
Epoch 3/100
11/11 [==============================] - 1s 58ms/step - loss: 0.5781 - mae: 0.5915 - val_loss: 1.1954 - val_mae: 0.7816
Epoch 4/100
11/11 [==============================] - 1s 53ms/step - loss: 0.5361 - mae: 0.5701 - val_loss: 1.1433 - val_mae: 0.7647
Epoch 5/100
11/11 [==============================] - 1s 55ms/step - loss: 0.5352 - mae: 0.5812 - val_loss: 1.0706 - val_mae: 0.7356
Epoch 6/100
11/11 [==============================] - 1s 53ms/step - loss: 0.4686 - mae: 0.5368 - val_loss: 1.0227 - val_mae: 0.7148
Epoch 7/100
11/11 [==============================] - 1s 64ms/step - loss: 0.4974 - mae: 0.5513 - val_loss: 0.9937 - val_mae: 0.7104
Epoch 8/100
11/11 [==============================] - 1s 55ms/step - loss: 0.